In [88]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/t4/gbgxnn9j3sj48qv4ydr2n_100000gn/T/ipykernel_3175/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [90]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [91]:
target = pd.read_csv("../data/kg_test.csv")

X_train = data['text']
y_train = data['label']

X_test = target['text']


## Data Preprocessing

In [92]:
import string
from nltk.corpus import stopwords

print(string.punctuation)
print(stopwords.words("english")[100:110])

from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [93]:
import re
from bs4 import BeautifulSoup


def remove_tags(text_series):
    cleaned_html_text = []
    for text in text_series:
        # Step 1: Remove inline JavaScript and CSS
        text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
        text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)

        # Step 2: Remove HTML comments
        text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

        # Step 3: Remove remaining HTML tags
        text = BeautifulSoup(text, 'html.parser').get_text()

        cleaned_html_text.append(text)

    return pd.Series(cleaned_html_text)

# Clean the text by applying the remove_tags function
X_train_cleaned = remove_tags(X_train)

print(X_train_cleaned.head())

0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
dtype: object


/var/folders/t4/gbgxnn9j3sj48qv4ydr2n_100000gn/T/ipykernel_3175/353167818.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
/var/folders/t4/gbgxnn9j3sj48qv4ydr2n_100000gn/T/ipykernel_3175/353167818.py:16: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [94]:
def clean_text(text_series):
    cleaned_special_text = []
    for text in text_series:
        # Remove special characters
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove single characters
        text = re.sub(r'\b\w\b', '', text)
        # Remove single characters from the start
        text = re.sub(r'^\b\w\b', '', text)
        # Substitute multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        # Remove prefixed 'b'
        text = text.replace("b'", "")
        # Convert to lowercase
        text = text.lower()
        # Add cleaned text to list
        cleaned_special_text.append(text)
    return pd.Series(cleaned_special_text)

X_train_super_clean = clean_text(X_train_cleaned)

print(X_train_super_clean)


0      dear sir strictly private business proposal am...
1                                                will do
2      noracheryl has emailed dozens of memos about h...
3      dear sirfmadamc know that this proposal might ...
4                                                    fyi
                             ...                        
995    so whats the latest it sounds contradictory an...
996    transfer of million pounds to youraccountmy na...
997    barb will call to explain are you back in the ...
998       yang on travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh just talked ...
Length: 1000, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [95]:
print(X_train_super_clean.head())
print(type(X_train_super_clean.iloc[0]))

0    dear sir strictly private business proposal am...
1                                              will do
2    noracheryl has emailed dozens of memos about h...
3    dear sirfmadamc know that this proposal might ...
4                                                  fyi
dtype: object
<class 'str'>


In [96]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')  
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

# Tokenize and remove stop words
X_train_super_clean = X_train_super_clean.apply(
    lambda text: ' '.join([word for word in word_tokenize(text) if word.lower() not in stop_words])
)

print(X_train_super_clean)

[nltk_data] Downloading package stopwords to /Users/sven/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sven/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0      dear sir strictly private business proposal mi...
1                                                       
2      noracheryl emailed dozens memos haiti weekend ...
3      dear sirfmadamc know proposal might surprise e...
4                                                    fyi
                             ...                        
995    whats latest sounds contradictory af decide sh...
996    transfer million pounds youraccountmy name mre...
997                       barb call explain back country
998          yang travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Length: 1000, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [97]:
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get part-of-speech (POS) tagging for more accurate lemmatization
def get_wordnet_pos(word):
    from nltk.corpus import wordnet
    from nltk import pos_tag
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,  # Adjective
        'N': wordnet.NOUN,  # Noun
        'V': wordnet.VERB,  # Verb
        'R': wordnet.ADV   # Adverb
    }
    return tag_dict.get(tag, wordnet.NOUN)  # Default to noun

# Tokenize the text and apply lemmatization
X_train_super_clean = X_train_super_clean.apply(
    lambda text: [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in word_tokenize(text)]
)

# Display the first few rows to verify the result
print(X_train_super_clean.head())

0    [dear, sir, strictly, private, business, propo...
1                                                   []
2    [noracheryl, email, dozen, memo, haiti, weeken...
3    [dear, sirfmadamc, know, proposal, might, surp...
4                                                [fyi]
dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [98]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

In [99]:
X_train_super_clean = X_train_super_clean.apply(lambda tokens: ' '.join(tokens))

In [100]:
X_train_clean_vect = vectorizer.fit_transform(X_train_super_clean)  # Fit the vectorizer to the training text data

In [101]:
ham_mask = y_train == 1  # Mask for "ham" messages
spam_mask = y_train == 0  # Mask for "spam" messages

In [102]:
# Sum word counts for "ham" and "spam"
ham_word_counts = np.array(X_train_clean_vect[ham_mask].sum(axis=0)).flatten()
spam_word_counts = np.array(X_train_clean_vect[spam_mask].sum(axis=0)).flatten()

# Map words to indices
words = vectorizer.get_feature_names_out()

# Create DataFrames for easy sorting
ham_df = pd.DataFrame({'word': words, 'count': ham_word_counts})
spam_df = pd.DataFrame({'word': words, 'count': spam_word_counts})

# Get top 10 words for "ham" and "spam"
top_ham_words = ham_df.sort_values(by='count', ascending=False).head(10)
top_spam_words = spam_df.sort_values(by='count', ascending=False).head(10)

print("Top 10 words in ham messages:")
print(top_ham_words)

print("\nTop 10 words in spam messages:")
print(top_spam_words)

Top 10 words in ham messages:
              word  count
10370        money    920
159        account    795
1800          bank    746
6709          fund    709
16645     transfer    529
2590      business    474
3750       contact    441
16575  transaction    416
3966       country    407
10190      million    386

Top 10 words in spam messages:
            word  count
12498         pm    115
2694        call    111
17968      would    106
17921       work    103
14186        say    103
15147      state    103
12746  president     94
16241       time     84
6905         get     82
12242    percent     77


## Extra features

In [103]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [77]:
# Get the feature names (vocabulary)
vocabulary = vectorizer.get_feature_names_out()
print(vocabulary)

['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']


In [78]:
word_counts = X_train_clean_vect.sum(axis=0).A1  # Convert the sparse matrix to a dense array
word_freq = zip(vocabulary, word_counts)
word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)

# Get the top 10 most frequent words
top_10_words = word_freq[:10]
print(top_10_words)

[('money', 924), ('account', 802), ('bank', 755), ('fund', 712), ('transfer', 531), ('business', 497), ('country', 458), ('contact', 447), ('transaction', 416), ('make', 410)]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the dataset with TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_super_clean)

# Print the shape of the vectorized dataset
print(X_train_tfidf.shape)

(1000, 18314)


## And the Train a Classifier?

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Create a Logistic Regression classifier
classifier = LogisticRegression(max_iter=1000)

In [86]:
X_val = target['text']

In [87]:
# Preprocess the validation data (apply the same preprocessing steps as with the training data)
X_val_super_clean = X_val.apply(lambda text: ' '.join([word for word in word_tokenize(text) if word.lower() not in stop_words]))
X_val_super_clean = X_val_super_clean.apply(lambda tokens: [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens])

# Train the classifier
classifier.fit(X_train_tfidf, y_train)

# Predict on the validation set (X_val_tfidf should be TF-IDF transformed text data for validation)
X_val_tfidf = tfidf_vectorizer.transform(X_val_super_clean)
y_pred = classifier.predict(X_val_tfidf)

# Evaluate the classifier
print(classification_report(y_val, y_pred))

AttributeError: 'list' object has no attribute 'lower'

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code